

Handcrafted Feature extraction - 
1. 



In [1]:
from google.colab import drive 
drive.mount('/gdrive')

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA   #Sentiment Intensity Analyser
import nltk
nltk.download('vader_lexicon')  


!pip install textatistic
from textatistic import Textatistic

Mounted at /gdrive


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
     |████████████████████████████████| 40 kB 4.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for textatistic: filename=textatistic-0.0.1-py3-none-any.whl size=29068 sha256=833802cbd33fb538a1b42063012ba1394abb5fef02b3ebb9feac009dc81ecca1
  Stored in directory: /root/.cache/pip/wheels/58/4a/1a/5ed2a089cbd2f98693b07221c4ab499c8c446e15b6123ba4a4
  Created wheel for pyhyphen: filename=PyHyphen-4.0.3-cp37-abi3-linux_x86_64.whl size=60312 sha256=b34b9069c5a84aa1ac04b872ce2ed5c04dc50ce3df999081775365d372ef1f73
  Stored in directory: /root/.cache/pip/wheels/4e/21/3e/e883a6e9969fdd074763213ddaeee0e781c359bbfda3fa435f
Successfully built textatistic pyhyphen
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the p

In [2]:
#counting length of sentences 

def sentLen(s):
  return len(s.split())



In [3]:
import pandas as pd 
import re

book_synopsis = pd.read_csv("/gdrive/My Drive/MSC_Thesis_Project/Data/bookSynopsis_ageRange_clean.csv") 

#the row 1190 has NAN for desc
#therefore giving the desc same as title 
#book_synopsis['Desc'][1190] = book_synopsis['Title'][1190]

t= ""
d=""
words = []
wordCount=0
avgWordLen = 0
sentences=[]
sentenceCount=0
avgSentLen=0

sentiments=[]
pos_sentiment=0.0
neu_sentiment=0.0
neg_sentiment=0.0

tr_delechall = 0.0
tr_flesch_score = 0.0
tr_fleschkincaid_score = 0.0
tr_gunningfog_score= 0.0
tr_smog_score= 0.0

desc_readability_scores= []

col_names=['Title','wordCount','avgWordLen','sentenceCount','avgSentLen','pos_sentiment','neu_sentiment','neg_sentiment',
           'tr_delechall','tr_flesch_score','tr_fleschkincaid_score','tr_gunningfog_score','tr_smog_score']
handcrafted_features = pd.DataFrame(columns = col_names)

for i in book_synopsis.index: 
    t = book_synopsis['Title'][i]
    d = book_synopsis['Desc'][i]
    words = d.split()
    #feature - number of words 
    wordCount= len(words)
    #feature - average length of words 
    avgWordLen = sum(len(word) for word in words) / len(words) 
    #feature - sentence count 
    d= re.sub('\.+', '.', d)      #replacing multiple '...' with '.'
    d= d.replace('\n','')         #replacing '\n' with ''
    if not (d.endswith('.')):     #always ensure that the 'desc' ends with '.'
       d= d+'.' 
    sentences= d.split(".")
    sentenceCount= len(sentences)
    #feature - average sentence length
    avgSentLen = sum(sentLen(s) for s in sentences) / sentenceCount   #formula avg = sum(len(sentence))/ total no.of sentences

    #Sentiment Analysis (positive, negative & neutral)
    sina = SIA()    #Sentiment Intensity Analyser
    sentiments =    sina.polarity_scores(' '.join(re.findall(r'\w+',d.lower())))
    
    pos_sentiment  = sentiments['pos']+1*(10**-6) 
    neu_sentiment  = sentiments['neu']+1*(10**-6)
    neg_sentiment  = sentiments['neg']+1*(10**-6)

   #Text readability scores -  
    
    desc_readability_scores = Textatistic(d).scores 
      

    tr_delechall           = desc_readability_scores['dalechall_score'] 	
    tr_flesch_score        = desc_readability_scores['flesch_score']
    tr_fleschkincaid_score = desc_readability_scores['fleschkincaid_score']
    tr_gunningfog_score	   = desc_readability_scores['gunningfog_score']
    tr_smog_score          = desc_readability_scores['smog_score']


    handcrafted_features = handcrafted_features.append({
                                 'Title'    : t,
                                 'wordCount': wordCount,
                                 'avgWordLen': avgWordLen,
                                 'sentenceCount': sentenceCount,
                                 'avgSentLen'   : avgSentLen,
                                 'pos_sentiment': pos_sentiment,
                                 'neu_sentiment': neu_sentiment,
                                 'neg_sentiment': neg_sentiment,
                                  'tr_delechall': tr_delechall,
                               'tr_flesch_score': tr_flesch_score,
                        'tr_fleschkincaid_score': tr_fleschkincaid_score,
                           'tr_gunningfog_score': tr_gunningfog_score,
                                 'tr_smog_score': tr_smog_score
                              }, ignore_index= True)
##
##Writing Handcrafted features to csv file handcrafted_features.csv
##
handcrafted_features.to_csv('/gdrive/My Drive/MSC_Thesis_Project/Data/handcrafted_features.csv')


In [4]:
#checking whether all rows match in both dataframes 
#book_synopsis & handcrafted_features - 
t= "" 
c = 0
for i in book_synopsis.index:
    t = handcrafted_features.iloc[i]['Title'] 
    if not(t==book_synopsis['Title'][i]):
        print(str(i), book_synopsis['Title'][i])
    else:
        c = c+1



print("c = "+str(c))

c = 1581


# Ref. https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/01-Basic-features-and-readability-scores.html#Readability-tests
# Readability Tests
